In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls -al drive/My\ Drive/data

total 1925451
-rw------- 1 root root 759780182 Nov  5 15:29 train_imgs_1.npy
-rw------- 1 root root 718965551 Nov  5 16:21 train_imgs_2.npy
-rw------- 1 root root 253260146 Nov  5 15:30 train_labels_1.npy
-rw------- 1 root root 239655269 Nov  5 16:21 train_labels_2.npy


In [14]:
train_imgs = np.array([])
train_labels = np.array([])

In [15]:
# load train (10 locations)
flag = False
for index_name in range(1,3):
    print("Iteration", index_name)
    if flag == False:
        train_imgs = np.load("drive/My Drive/data2/train_imgs_" + str(index_name) + ".npy")
        train_labels = np.load("drive/My Drive/data2/train_labels_" + str(index_name) + ".npy")
        flag = True
    else:
        train_imgs = np.concatenate([train_imgs, np.load("drive/My Drive/data2/train_imgs_" + str(index_name) + ".npy")], axis=0)
        train_labels = np.concatenate([train_labels, np.load("drive/My Drive/data2/train_labels_" + str(index_name) + ".npy")], axis=0)

Iteration 1
Iteration 2


In [16]:
train_imgs = train_imgs.astype("float32")
train_labels = train_labels.astype("float32")

### Preprocessing

In [17]:
patch_size = 200
threshold = 1000 #(200*200) / 10 # min number of 1 pixels needed
patches_imgs = np.array([])
patches_labels = np.array([])
found_patch = False

In [18]:
def count_pixels(img):
    return img[img > 0].shape[0]

In [19]:
# stack two images on channel axis and compute difference between label images
indices = [0,25,50,74,98,122,144,169,193,218,242,266]

for k in range(len(indices)-1):
    for t1 in range(indices[k], indices[k+1]):
        if t1%10==0:
            print(t1)
        for t2 in range(t1, indices[k+1]):
            pair_img = np.concatenate([train_imgs[t2], train_imgs[t1]], axis=-1)
            new_label = (train_labels[t2]) - (train_labels[t1])
            new_label[new_label<0] = 0

            if new_label[(new_label < 255) & (new_label > 0)].shape[0] > 0:
                print(t2,t1)
            for i in range(0, 1023-patch_size, patch_size):
                for j in range(0, 1023-patch_size, patch_size):
                    img = new_label[i:i+patch_size, j:j+patch_size]
                    if count_pixels(img) > threshold: # keep image
                        if not found_patch:
                            patches_imgs = np.expand_dims(pair_img[i:i+patch_size, j:j+patch_size,:], axis=0)
                            patches_labels = np.expand_dims(img, axis=0)
                            found_patch = True
                        else:
                            patches_imgs = np.concatenate([patches_imgs, np.expand_dims(pair_img[i:i+patch_size, j:j+patch_size,:], axis=0)], axis=0)
                            patches_labels = np.concatenate([patches_labels, np.expand_dims(img, axis=0)], axis=0)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260


In [20]:
patches_imgs.shape, patches_labels.shape

((1830, 200, 200, 6), (1830, 200, 200))

In [21]:
# Crop images to img_size x img_size
train_imgs = np.divide(patches_imgs, 255.)
train_labels = np.divide(patches_labels, 255)

# Expand dims for compatibility with the network
train_labels = np.expand_dims(train_labels, axis=-1)

In [22]:
train_imgs.shape, train_labels.dtype

((1830, 200, 200, 6), dtype('float32'))

In [23]:
train_labels.shape

(1830, 200, 200, 1)

In [24]:
np.save("drive/My Drive/data2/imgs_final_dataset.npy", train_imgs)
np.save("drive/My Drive/data2/labels_final_dataset.npy", train_labels)